In [ ]:
!pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.8 MB/s eta 0:00:00


In [ ]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=100, shuffle=True, noise=0.3, random_state=43)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
    model = keras.Sequential()
    model.add(
        layers.Dense(
            # Define the hyperparameter.
            units=hp.Int("units", min_value=32, max_value=512, step=32),
            activation="relu",
        )
    )
    model.add(layers.Dense(1, activation="linear"))
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.AUC(from_logits=True)],
        #metrics=["accuracy"],
    )
    return model

In [ ]:
import keras_tuner
import numpy as np
import io
import copy
from sklearn import model_selection
from keras_tuner.engine import tuner_utils

class CVTuner(keras_tuner.engine.tuner.Tuner):
    def run_trial(self, trial, x, y, *args, **kwargs):
        
        model_checkpoint = tuner_utils.SaveBestEpoch(
            objective=self.oracle.objective,
            filepath=self._get_checkpoint_fname(trial.trial_id),
        )
        original_callbacks = kwargs.pop("callbacks", [])

        # Run the training process multiple times.
        obj_list = []
        histories = []
        """ --------- Replace self.executions_per_trial by number of K-flod executions --------- """
        cv = model_selection.KFold(5)
        for execution, (train_indices, test_indices) in enumerate( cv.split(x) ):
            X_train, X_test = x[train_indices], x[test_indices]
            y_train, y_test = y[train_indices], y[test_indices]
            """ -------------------------------------------------------------------------------- """
            
            copied_kwargs = copy.copy(kwargs)
            callbacks = self._deepcopy_callbacks(original_callbacks)
            self._configure_tensorboard_dir(callbacks, trial, execution)
            callbacks.append(tuner_utils.TunerCallback(self, trial))
            # Only checkpoint the best epoch across all executions.
            callbacks.append(model_checkpoint)
            copied_kwargs["callbacks"] = callbacks
            
            """ ---------- add more argument when calling self._build_and_fit_model ------------ """
            obj_value = self._build_and_fit_model(trial, x=X_train, y=y_train, validation_data=(X_test, y_test), *args, **copied_kwargs)
            #hp = trial.hyperparameters
            #model = self._try_build(hp)
            #obj_value = self.hypermodel.fit(hp, model, x=X_train, y=y_train, validation_data=(X_test, y_test), *args, **kwargs)
            #tuner_utils.validate_trial_results(obj_value, self.oracle.objective, "HyperModel.fit()")
            """ ---------------------------------------------------------------------------------"""
            
            """ by adding each training logs into histories as the return, BaseTuner will takes care
             of computing average object metric scores over all executions in this trail. """
            histories.append(obj_value)
            """ ----------- below code is to verify the behavior of averaging score ------------ """
            if self.oracle.objective.direction == "max":
                obj_list.append( max(obj_value.history[self.oracle.objective.name]) )
            else:
                obj_list.append( min(obj_value.history[self.oracle.objective.name]) )
        tf.print(np.mean(obj_list))
        return histories

buffer = io.BytesIO()

tuner = CVTuner(
    hypermodel = build_model,
    oracle = keras_tuner.oracles.BayesianOptimization(
        objective = keras_tuner.Objective("val_auc", direction="max"), 
        #objective = "val_loss",
        max_trials = 5
        ),
    overwrite=True,
    directory = buffer,
    project_name = 'simple_proj'
    )

tuner.search(X, y, batch_size=64, epochs=5, verbose=0) # no need for validation_data=(X_test, y_test)

0.91474289894104
0.9215472221374512
0.9347020745277405
0.8733270287513732
0.9162164330482483


In [ ]:
tuner.results_summary()

Results summary
Results in <_io.BytesIO object at 0x7f9a13828b30>/simple_proj
Showing 10 best trials
Trial summary
Hyperparameters:
units: 352
Score: 0.9347020745277405
Trial summary
Hyperparameters:
units: 416
Score: 0.9215472221374512
Trial summary
Hyperparameters:
units: 256
Score: 0.9162164330482483
Trial summary
Hyperparameters:
units: 480
Score: 0.91474289894104
Trial summary
Hyperparameters:
units: 32
Score: 0.8733270287513732


In [ ]:
best_hps = tuner.get_best_hyperparameters(2)
model = build_model(best_hps[0])
model.build(input_shape=(None, 2))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 352)               1056      
                                                                 
 dense_3 (Dense)             (None, 1)                 353       
                                                                 
Total params: 1,409
Trainable params: 1,409
Non-trainable params: 0
_________________________________________________________________
